# ※ AWS는 적재, 조회 등에 비용이 발생하니 최종 코드가 아닌 한 local에서 TEST!

In [ ]:
# aws 계정 로그인 셀
import pymysql
import pandas as pd # 불요. 출력값 보기용.

# 계정 정보 파일에서 필요 인자 딕셔너리로 생성
with open('secret_local_db.config') as f:
    res = dict(map(lambda x: x.replace('\n','').split('='), f.readlines()))
    for i, v in res.items():
        if v.isdigit(): res[i] = int(v)
# 로그인 위한 인자명 출력
print(res.keys())
# 딕셔너리 언패킹하여 인자 값 할당 후 서버 연동
remote = pymysql.connect(**res)

In [ ]:
# db 및 유저 생성

# with로 여는 것은 cursor가 여러 개 열리는 것을 예방하기 위함.
with remote.cursor() as cur:
    # db명 설정 및 생성
    dbname = 'db4_6'
    # cur.execute(f'create database {dbname}')
    cur.execute('show databases')
    # fetchall(): 출력문 전체 가져오기
    # print(cur.fetchall())
    print(f'{"  show databases  ":=^80}')
    print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['database_name']))
    
    # # user 생성 및 권한 지정
    # username = 'tmp_01'
    # password = 'tmp_1234'   # 연습용이 아닌 실제 생성 user는 secret 파일을 통해 인자 입력 필요
    # cur.execute('use mysql')
    # # user 생성, %는 외부 접속 사용자라는 의미.
    # cur.execute(f'create user \'{username}\'@\'%\' identified by \'{password}\'')
    # # 특정 db 권한 전부(*) 부여
    # cur.execute(f'grant all on {dbname}.* to \'{username}\'@\'%\'')
    # # 특정 user 권한 확인
    # cur.execute(f'show grants for \'{username}\'@\'%\'')
    # # print(cur.fetchall())
    # print(f'''{"  show grants for '{username}'@'%'  ":=^80}''')
    # print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['grant users']))

In [3]:
# # db 및 유저 삭제

# with remote.cursor() as cur:
#     # user 권한 제거 및 user 삭제
#     username = 'tmp_01'
#     cur.execute('use mysql')
#     # user 테이블의 host, user 컬럼 조회
#     cur.execute('select host, user from user')
#     # print(cur.fetchall())
#     print(f'{"  select host, user from user  ":=^80}')
#     print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['host', 'user']))
    
#     # 특정 db 권한 전부(*) 제거
#     cur.execute(f'revoke all on {dbname}.* from \'{username}\'@\'%\'')
#     # 특정 user 권한 확인
#     cur.execute(f'show grants for \'{username}\'@\'%\'')
#     # print(cur.fetchall())
#     print(f'''{"  show grants for '{username}'@'%'  ":=^80}''')
#     print(pd.DataFrame([_ for _ in cur.fetchall()]))

#     # 특정 user 삭제
#     cur.execute(f'drop user \'{username}\'@\'%\'')
#     # user 제거 확인
#     cur.execute('select host, user from user')
#     # print(cur.fetchall())
#     print(f'{"  select host, user from user  ":=^80}')
#     print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['host', 'user']))


#     # db 삭제
#     cur.execute('show databases')
#     # print(cur.fetchall())
#     print(f'{"  show databases  ":=^80}')
#     print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['database_name']))
    
#     dbname = 'db4_6'
#     cur.execute(f'drop database {dbname}')
#     # db 삭제 확인
#     cur.execute('show databases')
#     # print(cur.fetchall())
#     print(f'{"  show databases  ":=^80}')
#     print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['database_name']))

In [ ]:
with remote.cursor() as cur:
    cur.execute('show tables')
    print(f'{"  show tables  ":=^80}')
    print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['tables']))

In [ ]:
sql_file = 'Elementary_ERD.sql'
# sql_file = 'Elementary_ERD.txt'

with remote.cursor() as cur:
    # 파일에서 '\ufeff'가 읽힐 경우 encoding하거나 replace로 제거
    with open(sql_file, 'r', encoding='utf-8-sig') as f :
        # split(';')이기에 마지막에 ['']가 존재하여 [:-1]로 슬라이싱
        commands = f.read().split(';')[:-1]
    for command in commands:
        cur.execute(command.strip())

In [2]:
import json

with remote.cursor() as cur:
    # category 불러오기
    try:
        with open('category.json', 'r') as f:
            category = json.load(f)
    except:
        print("category.json don't exist or path is worng.")

    # category 확인 및 적재
    done_list = []
    error_list = []
    try:
        cur.execute('select count(*) from category')
        if cur.fetchall()[0][0]==145:
            print('='*50)
            print('already all record is loaded on CATEGORY table')
            pass
        else:
            for platfrom in category.keys():
                for cat_1 in category[platfrom].keys():
                    for name, id in category[platfrom][cat_1].items():
                        try:
                            cur.execute(f"insert into category values({id}, '{cat_1}','{name}', '{platfrom}')")
                            done_list.append([id, cat_1, name, platfrom])
                        except:
                            error_list.append([id, cat_1, name, platfrom])
                            print(f"already values({id}, '{cat_1}','{name}', '{platfrom}' exist")
            print('='*50)
            print(f'done_tasl: {len(done_list)}, error_task: {len(error_list)}')
    except:
        print("make tables first!")

# DML은 별도 commit 필요!
remote.commit()
print('task is done!')

In [ ]:
# naver news 전처리
import pandas as pd
import pickle
import re

with open('secret_naver_news_data.pkl', 'rb') as f:
    df_naver = pd.DataFrame(pickle.load(f))

ch_df = df_naver[df_naver[1].apply(lambda x: False if x.find('@')==-1 else True)][1].str.split('@').str[0].apply(lambda x: ' '.join(re.split(r'[(, )]', x)[:-1]))
df_naver.loc[ch_df.index,1] = ch_df
df_naver[9] = df_naver[9].apply(lambda x: ' '.join([_ for _ in x if '\t' not in _ and '\n' not in _]))
df_naver[12] = '세계'
df_naver = df_naver[[12,5,8,2,0,1,3,4,9,11,7]]
df_columns = ['cat1_name', 'cat2_name', 'platform_name', 'title', 'press', 'writer', 'date_upload', 'date_fix', 'content', 'sticker', 'url']
df_naver.columns = df_columns

# 결측값은 모두 None으로 변경
df_naver.loc[df_naver[df_naver['date_fix']=='null'].index, 'date_fix'] = None
df_naver.loc[df_naver[df_naver['writer']=='null'].index, 'writer'] = None

In [ ]:
# naver news 전처리
import pandas as pd
import pickle
import re

with open('secret_daum_news_data.pkl', 'rb') as f:
    df_daum = pd.DataFrame(pickle.load(f))

df_columns = ['cat1_name', 'cat2_name', 'platform_name', 'title', 'press', 'writer', 'date_upload', 'date_fix', 'content', 'sticker', 'url']
classified = {1411: '아시아/대양주', 1412: '미국/아메리카', 1413: '유럽', 1414: '중동/아프리카', 1415: '국제일반', 1416: '영어뉴스', 1417: '해외화제', 1418: '일본', 1419: '중국'}

df_daum[11] = '국제'
df_daum = df_daum[[11,6,8,2,0,1,3,4,9,10,7]]
df_daum[6] = df_daum[6].apply(lambda x: classified.get(int(x)))
df_daum[1] = df_daum[1].apply(lambda x: None if str(x).find('입력 <span')==0 else x)
df_daum[3] = [datetime(int(__[0]), int(__[1]), int(__[2]), int(__[3].split(':')[0]), int(__[3].split(':')[1])).strftime('%Y-%m-%d %H:%M:%S') for __ in [_.split('. ') for _ in df_daum[3]]]
df_daum[4] = None
df_daum.columns = df_columns

In [ ]:
# column 이름 일치 확인
def insert_news(remote, df):
    df_columns = ['cat1_name', 'cat2_name', 'platform_name', 'title', 'press', 'writer', 'date_upload', 'date_fix', 'content', 'sticker', 'url']
    if sum(~(df_columns==df.columns)):
        raise Exception(f"columns' name dont matched!!\nmake columns' name like {df_columns}")

    # platform_name 변환 및 cat2_id 할당
    with open('category.json', 'r') as f:
                SUB_CATEGORY_DICT = json.load(f)
                
    df['platform_name'] = [platform if platform in ("네이버", "다음") else "네이버" if platform.upper()=="NAVER" else "다음" if platform.upper()=="DAUM" else None for platform in df['platform_name']]
    if df['platform_name'].isna().any():
        raise Exception('some platform_name rows wrong!!')
    df['cat2_id'] = [SUB_CATEGORY_DICT[_[0]][_[1]][_[2]] for _ in df[['platform_name', 'cat1_name', 'cat2_name']].values]
    df_columns = ['cat2_id']+df_columns
    df = df[df_columns]
    df.drop(columns=['cat1_name', 'cat2_name', 'platform_name'], inplace=True)

    # sticker json 따옴표 변경
    if not sum(df['sticker'].apply(str).str.count('"')):
        df['sticker'] = df['sticker'].apply(json.dumps)

    with remote.cursor() as cur:
        my_query = "insert ignore into NEWS(cat2_id, title, press, writer, date_upload, date_fix, content, sticker, url) values(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cur.executemany(my_query, df.values.tolist())
    remote.commit()
insert_news(remote, df_daum)
insert_news(remote, df_naver)

In [ ]:
# naver comment 전처리
import pandas as pd
import pickle
import re
from datetime import datetime

with open('secret_naver_news_data.pkl', 'rb') as f:
    df_naver = pd.DataFrame(pickle.load(f))
df = df_naver[[10,7]]
df.columns = ['comment', 'url']
df = df[df.comment.apply(len)!=0]

In [ ]:
tmp_list = []
with remote.cursor() as cur:
    my_query = "select id, url from news where url=%s"
    for v in df['url'].values:
        cur.execute(my_query, v)
        tmp_list.extend(cur.fetchall())

tmp_list = pd.DataFrame(tmp_list, columns=['news_id', 'url'])
df = pd.merge(df, tmp_list, 'left', 'url').explode('comment').reset_index(drop=True)
df.head()

In [ ]:
trash, df['comment'], df['user_id'], df['user_name'], df['date_upload'], df['date_fix'], df['good_cnt'], df['bad_cnt'] = zip(*df.comment.values)
del trash
df['date_upload'] =  df['date_upload'].str.split('+').str[0]
df['date_upload'] = [' '.join(date__[:-5].split('T')) for date__ in df['date_upload']]
df['date_fix'] = [' '.join(date__[:-5].split('T')) for date__ in df['date_fix']]
df = df[~df.user_id.isna()].reset_index(drop=True)
df_columns = ['news_id', 'user_id', 'user_name', 'comment', 'date_upload', 'date_fix', 'good_cnt', 'bad_cnt']
df = df[df_columns]

In [ ]:
user_df = df.groupby(['user_id', 'user_name']).count().reset_index()[['user_id', 'user_name']]
with remote.cursor() as cur:
    my_query = "insert ignore into USER(user_id, user_name) values(%s, %s)"
    cur.executemany(my_query, user_df.values.tolist())
remote.commit()

In [ ]:
# get user_id
tmp_list = []
with remote.cursor() as cur:
    my_query = "select id, user_id from user where user_id=%s"
    for v in user_df.user_id.values:
        cur.execute(my_query, v)
        tmp_list.extend(cur.fetchall())
tmp_list = pd.DataFrame(tmp_list, columns=['id', 'user_id'])

In [ ]:
df = pd.merge(df, tmp_list.drop_duplicates('user_id').reset_index(drop=True), 'left', 'user_id').reset_index(drop=True)
df_columns.pop(2)
df = df.drop(columns='user_id').rename(columns={'id': 'user_id'})[df_columns]

In [ ]:
with remote.cursor() as cur:
    my_query = "insert ignore into COMMENT(news_id, user_id, comment, date_upload, date_fix, good_cnt, bad_cnt) values(%s, %s, %s, %s, %s, %s, %s)"
    cur.executemany(my_query, df.values.tolist())
remote.commit()